### Import libraries

In [ ]:
# Installing split-folders for dataset split process
!pip install split-folders

In [ ]:
# Importing all the required libraries
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
from PIL import Image
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import splitfolders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Setting the random seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### PREPARING THE DATA

In [ ]:
# TRAINING VARIABLES
# Learning rate 
lr = 1e-3   # 0.0005 - 0.000001
# Batch size
batch_size = 64

# Dataset folders, must include dataset into same folder as the notebook to run
dataset_dir = './dataset'
training_dir = os.path.join(dataset_dir, 'training')

# Listing all the labels
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']


### Data Loader

In [ ]:
# REMOVE COMMENT ONLY ON FIRST RUN, ONLY ONE SPLIT NEEDED! 
# Dividing dataset into training, validation and test to avoid overfitting..
splitfolders.ratio(training_dir, output="output", seed=seed, ratio=(.7,.2,.1), group_prefix=None)


# Using ImageDataGenerator to exploit folder organization for reading

train_dir = "output/train"
test_dir = "output/test"
valid_dir = "output/val"

# Initializing the generators, training one will have data augmentation options
# to generalize the results more
train_data_gen = ImageDataGenerator(rotation_range=30, 
                                    width_shift_range=15,
                                    height_shift_range=15,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    fill_mode='nearest')
valid_data_gen = ImageDataGenerator()
test_data_gen = ImageDataGenerator()

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=train_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, 
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed,
)
valid_gen = valid_data_gen.flow_from_directory(directory=valid_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None,
                                               batch_size=batch_size,
                                               shuffle=False,
                                               seed=seed,)
test_gen = test_data_gen.flow_from_directory(directory=test_dir,
                                              target_size=(256,256),
                                              color_mode='rgb',
                                              classes=None, 
                                              batch_size=batch_size,
                                              shuffle=False,
                                              seed=seed)

In [ ]:
# Sanity check
print("Assigned labels")
print(train_gen.class_indices)
print()
print("Target classes")
print(train_gen.classes)

### TRANSFER LEARNING MODEL

In [ ]:
# Download and visualizing the VGG16 model
supernet = tfk.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3)
)

supernet.summary()

tfk.utils.plot_model(supernet)

In [ ]:
def build_model(input_shape):
    # Use the supernet as feature extractor
    supernet.trainable = False
    inputs = tfk.Input(shape=input_shape)
    x = supernet(inputs)
    
    # Adding GAP + dropout for better feature extraction and less overfitting
    x = tfkl.GlobalAveragePooling2D(name='GlobalPooling')(x)
    x = tfkl.Dropout(0.3, seed=seed, name='GlobalPoolingDropout')(x)
    
    # Two dense layers with ReLU activation just like the real VGG16
    x = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(x)
    x = tfkl.Dense(units=512, name='Classifier2', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(x)

    # Output layer for classification
    outputs = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(x)
    
    # Connect input and output through the Model class
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
    
    # Compiling the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
    
    # Return the model
    return model

In [ ]:
input_shape = (256, 256, 3)
epochs = 50

model = build_model(input_shape)

# Double checking the structure. In first training, no layer is trainable
model.get_layer('vgg16').trainable = False
for i, layer in enumerate(model.get_layer('vgg16').layers):
       print(i, layer.name, layer.trainable)
model.summary()

In [ ]:
# Suppressing WARNING logs during the training
tf.get_logger().setLevel('ERROR') 

# Train the model
history = model.fit(
    train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
                 tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5)]
).history

In [ ]:
model.save('transfer_learning_vgg16_no_ft')

## FINE TUNING THE MODEL

In [ ]:
# Make all layers trainable at first
model.get_layer('vgg16').trainable = True

# Freeze all layers except last block
for i, layer in enumerate(model.get_layer('vgg16').layers[:14]):
      layer.trainable = False
        
# Recap of the network
for i, layer in enumerate(model.get_layer('vgg16').layers):
       print(i, layer.name, layer.trainable)
        
model.summary()

# Recompiling the model and setting a low learning rate
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
# Re-train the model
history = model.fit(
    train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
                 tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5)]
).history

In [ ]:
model.save('vgg16_tl_ft')

In [ ]:
# Plotting loss and accuracy of the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Testing the model on the test set 
model.evaluate(test_gen)